# The Sims-Flanagan gradients
This notebook is the result of a few days of blood shed with manual derivatives assembly. 
It benchmarks a python version of the computation of a sims-flanagan forward only leg and the same done using heyoka.

In [1]:
import heyoka as hy
import numpy as np
import pykep as pk

from numba import njit

## Problem data
We define here a specific test instance.

In [2]:
# Problem data
mu = pk.MU_SUN
max_thrust = np.random.random()
isp = 3000.0*np.random.random()
veff = isp * 9.80665

# Initial state
ms = 1500.0 * np.random.random()
rs = np.array([1, 0.1, -0.1]) * pk.AU
vs = np.array([0.2, 1, -0.2]) * pk.EARTH_VELOCITY

# Variables
nseg = 5
tof = 324.0 * pk.DAY2SEC * (np.random.random() + 1)
throttles = (0.5-np.random.random(3*nseg))

# C++ test
ms = 1500.0
rs = np.array([1, 0.1, -0.1]) * pk.AU
vs = np.array([0.2, 1, -0.2]) * pk.EARTH_VELOCITY
throttles = np.array([0.010, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019, 0.02, 0.021, 0.022, 0.023, 0.024])
throttles_bck = np.array([0.022, 0.023, 0.024, 0.019, 0.02, 0.021, 0.016, 0.017, 0.018, 0.013, 0.014, 0.015,0.010, 0.011, 0.012])
tof = 324.0 * pk.DAY2SEC
max_thrust = 0.01
isp = 3000
veff = isp * 9.80665


## A single shooting
This code is thought in forward mode, but backwards will be similar

In [3]:
# The dynamics
def dyn(r, v, mu):
    R3 = np.linalg.norm(r) ** 3
    f = [v[0], v[1], v[2], -mu / R3 * r[0], -mu / R3 * r[1], -mu / R3 * r[2]]
    return np.array(f).reshape(-1, 1)

In [4]:
# This is the main computation ported to C++. Manual assembly of the gradients.
def gradients_multiple_impulses(xs, ys, zs, vxs, vys, vzs, ms, throttles, tof, veff): 
    nseg = int(throttles.shape[0] / 3)
    c = max_thrust * tof / nseg
    a = 1. / veff
    dt = tof / nseg
    m0=ms
    # We define the indepedent variables
    u = []
    un = []
    du = []
    m = [m0]
    for i in range(nseg):
        u.append(np.array(throttles[i*3:i*3+3]).reshape(1,3))
        du.append(np.zeros((3, 3*nseg+2)))
        du[i][:, 3*i:3*i+3] = np.eye(3)
    dm = [np.hstack((np.zeros((1, 3*nseg)), np.eye(1), np.zeros((1, 1))))]
    dtof = np.hstack((np.zeros((1, 3*nseg+1)), np.eye(1)))
    Dv = []
    dDv = []

    # 1 - We compute the mass schedule and related quantities
    for i in range(nseg):
        Dv.append(c / m[i] * u[i])
        un = np.sqrt(u[i][0][0]**2+u[i][0][1]**2+u[i][0][2]**2)
        Dvn = c / m[i] * un
        dDv.append(c / m[i] * du[i] - c / m[i]**2 * u[i].T@dm[i] + max_thrust / m[i] * u[i].T@dtof / nseg)
        dDvn = c / m[i] / un * u[i] @ du[i] - c / m[i]**2 * un * dm[i] + max_thrust / m[i] * un * dtof / nseg
        m.append(m[i] * np.exp(-Dvn * a))
        dm.append(-m[i+1] * a * dDvn + np.exp(-Dvn * a) * dm[i])

    M=[]
    f=[]
    rv_it = [[xs, ys,zs],[vxs,vys,vzs]]
    tofs = [dt]*(nseg+1)
    tofs[0]/=2.
    tofs[-1]/=2.

    # 2 - We compute the state transition matrices
    for i in range(nseg+1):
        rv_it, M_it = pk.propagate_lagrangian(
            rv=rv_it, tof=tofs[i], mu=mu, stm=True)
        M.append(M_it)
        #We compute f (before impulse)
        r = rv_it[0]
        f.append(dyn(r, rv_it[1], mu))
        #And add the impulse if needed
        if i<nseg:
            rv_it[1] = [a + b for a, b in zip(rv_it[1], Dv[i].flatten())]


    # 3 - We assemble the gradients
    Iv = np.diag((0, 0, 0, 1, 1, 1))[:, 3:]

    # Mc will contain [Mn@..@M0,Mn@..@M1, Mn]
    Mc=[0]*(nseg+1)
    Mc[-1] = M[-1]
    for i in range(1, len(M)):
        Mc[-1-i] = Mc[-1-i+1]@M[-1-i]

    # grad_tof 
    grad_tof = 0.5 * f[-1]
    for i in range(nseg-1):
        grad_tof+= Mc[i+2]@f[i+1]
    grad_tof+= 0.5 * Mc[1]@f[0]
    grad_tof/=nseg
    for i in range(nseg):
        grad_tof+=Mc[i+1]@Iv@dDv[i][:, -1:]

    # grad u
    grad_u=0
    for i in range(nseg):
        grad_u += Mc[i+1]@Iv@dDv[i][:, :-2]

    # grad ms
    grad_ms=0
    for i in range(nseg):
        grad_ms += Mc[i+1]@Iv@dDv[i][:, -2:-1]

    # grad xs
    grad_xs = Mc[0]

    # Assembling te return value
    # grad will contain the gradient of the final posvelm with respect to the throttles and the tof
    grad = np.hstack((grad_u, grad_tof))
    grad = np.vstack((grad, dm[-1][:,:-1]))
    grad[-1,-1] = dm[-1][:,-1][0]

    # grad_ic will contain the gradient of the final posvelm with respect to the initial posvelm
    grad_ic = np.vstack((grad_xs, np.zeros((1,6))))
    grad_ic = np.hstack((grad_ic, np.zeros((7,1))))
    grad_ic[:6,-1:] = grad_ms
    grad_ic[-1,-1] = dm[-1][:,-2][0]
    return (grad, grad_ic)


In [6]:
def gradients_fwd(xs, ys, zs, vxs, vys, vzs, ms, throttles_fwd, tof, veff):
    return gradients_multiple_impulses(xs, ys, zs, vxs, vys, vzs, ms, throttles_fwd, tof, veff)

def gradients_bck(xf, yf, zf, vxf, vyf, vzf, mf, throttles_bck, tof, veff):
    grad, grad_ic = gradients_multiple_impulses(xf, yf, zf, -vxf, -vyf, -vzf, mf, -throttles_bck, tof, -veff)
    grad_ic[3:6,:]*=-1
    grad_ic[:,3:6]*=-1
    grad[:, :-1]*=-1
    grad[3:6, :]*=-1

    return (grad, grad_ic)

In [7]:
%%timeit
grad, grad_ic = gradients_fwd(rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms, throttles, tof, veff)

567 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
grad, grad_ic = gradients_fwd(rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms, throttles, tof, veff)

In [9]:
grad_ic

array([[-1.97136415e+01, -3.03053217e+00,  2.11726965e+00,
        -2.97384315e+07, -1.09791311e+08,  2.24140294e+07,
         5.82966847e+03],
       [ 1.40595638e+01,  3.62733934e+00, -1.92475642e+00,
         2.99739124e+07,  7.67944235e+07, -1.69455639e+07,
        -2.62781138e+04],
       [-1.01994457e+00, -5.29080681e-01, -4.21923232e-01,
        -3.25006102e+06, -5.52482843e+06, -8.74094818e+05,
        -5.98621920e+03],
       [-4.45929416e-06, -6.62436240e-07,  5.03477699e-07,
        -6.75253996e+00, -2.44188095e+01,  4.96298674e+00,
         2.47113110e-04],
       [-2.69183825e-06, -6.79261012e-07,  3.84169599e-07,
        -5.79422489e+00, -1.41422541e+01,  2.90702927e+00,
         2.35440983e-03],
       [ 8.76526389e-07,  2.18488935e-07,  1.04025433e-07,
         1.54337030e+00,  4.43044250e+00, -1.82831237e+00,
         8.77429582e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         9.9999999

# Test with heyoka
We compile the same function bu using heyoka

In [10]:
def propagate_lagrangian_heyoka(pos_0, vel_0, mu, tof):
    x0, y0, z0 = pos_0
    vx0, vy0, vz0 = vel_0
    v02 = vx0**2 + vy0**2 + vz0**2
    r0 = hy.sqrt(x0**2 + y0**2 + z0**2)
    eps = v02 * 0.5 - mu / r0
    a = -mu / (2.0 * eps)

    sigma0 = np.dot(pos_0, vel_0) / np.sqrt(mu)
    s0 = sigma0 / hy.sqrt(a)
    c0 = 1.0 - r0 / a

    n = hy.sqrt(mu / (a * a * a))
    DM = n * tof

    DE = hy.kepDE(s0, c0, DM)

    # Compute cos(DE) and sin(DE).
    cDE = hy.cos(DE)
    sDE = hy.sin(DE)

    r = a + (r0 - a) * cDE + sigma0 * hy.sqrt(a) * sDE

    F = 1.0 - a / r0 * (1.0 - cDE)
    G = a * sigma0 / np.sqrt(mu) * (1.0 - cDE) + r0 * hy.sqrt(a / mu) * sDE
    Ft = -hy.sqrt(mu * a) / (r * r0) * sDE
    Gt = 1 - a / r * (1.0 - cDE)

    pos = F * pos_0 + G * vel_0
    vel = Ft * pos_0 + Gt * vel_0

    return [pos, vel]

def gradients_fwd_heyoka(xs, ys, zs, vxs, vys, vzs, ms, throttles_vars, tof, vef):
    nseg = int(len(throttles) / 3)
    dt = tof / nseg
    c = max_thrust * dt
    
    pos = np.array([xs, ys, zs])
    vel = np.array([vxs, vys, vzs])
    m=ms

    for i in range(nseg):
        ux, uy, uz = throttles_vars[3*i:3*i+3]
        if i == 0:
            pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt/2)
        else:
            pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt)  
        vel = vel + np.array([ux, uy, uz]) * c / m
        m = m * hy.exp(-hy.sqrt(ux**2+uy**2+uz**2) * c / veff / m)
    # 4
    pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt/2)
    retval = np.hstack([pos, vel, m])
    return retval

def gradients_bck_heyoka(xf, yf, zf, vxf, vyf, vzf, mf, throttles_vars, tof, veff):
    nseg = int(len(throttles) / 3)
    dt = tof / nseg
    c = max_thrust * dt
    
    pos = np.array([xf, yf, zf])
    vel = np.array([-vxf, -vyf, -vzf])
    m=mf

    for i in range(nseg):
        ux, uy, uz = throttles_vars[3*i:3*i+3]
        if i == 0:
            pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt/2)
        else:
            pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt)  
        vel = vel - np.array([ux, uy, uz]) * c / m
        m = m * hy.exp(hy.sqrt(ux**2+uy**2+uz**2) * c / veff / m)
    # 4
    pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt/2)
    retval = np.hstack([pos, -vel, m])
    return retval

In [11]:
# The symbolic variables
xs_var, ys_var, zs_var = hy.make_vars("xs", "ys", "zs")
vxs_var, vys_var, vzs_var = hy.make_vars("vxs", "vys", "vzs")
ms_var, = hy.make_vars("ms")
tof_var, = hy.make_vars("t")
throttles_symbols = []
for i in range(nseg):
    throttles_symbols.extend(["ux"+str(i), "uy"+str(i), "uz"+str(i)])
throttles_vars = hy.make_vars(*throttles_symbols)

In [12]:
# Here we compile the two functions for the two gradients
xf = gradients_fwd_heyoka(xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, throttles_vars, tof_var, veff)

dtens_ic = hy.diff_tensors(xf,
                    diff_args=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var],
                    diff_order=1
                    )
dtens = hy.diff_tensors(xf,
                    diff_args=[*throttles_vars, tof_var],
                    diff_order=1
                    )
jac_ic = dtens_ic.jacobian   
jac = dtens.jacobian   

d_cf = hy.make_cfunc(jac.flatten(),
                       # Specify the order in which the input
                       # variables are passed to the compiled
                       # function.
                       vars=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, *throttles_vars, tof_var])

d_cf_ic = hy.make_cfunc(jac_ic.flatten(),
                       # Specify the order in which the input
                       # variables are passed to the compiled
                       # function.
                       vars=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, *throttles_vars, tof_var])


In [13]:
%%timeit
analytical_grad_ic = d_cf_ic([rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms] + throttles.tolist() + [tof])
analytical_grad = d_cf([rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms] + throttles.tolist() + [tof])

13.6 µs ± 33.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [14]:
analytical_grad_ic = d_cf_ic([rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms] + throttles.tolist() + [tof])
analytical_grad = d_cf([rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms] + throttles.tolist() + [tof])

In [15]:
## Lets check the difference in the values computed is zero / small

In [16]:
(analytical_grad.reshape(7,-1)-grad)/grad

array([[ 1.78865446e-15, -1.94790061e-15, -2.61265240e-15,
        -1.20454473e-15, -1.96695803e-15, -2.42464804e-15,
        -1.07397152e-15, -1.71815840e-15,  6.51236793e-16,
        -6.63974835e-16,  5.03035721e-15, -7.28345697e-15,
         3.85875624e-16, -1.97346396e-14, -5.62196705e-15,
        -1.61822769e-15],
       [-7.38056674e-15,  4.19161735e-16,  6.98743897e-16,
         1.34570508e-15,  1.09918249e-15, -2.08498259e-16,
         1.57278892e-15,  1.40231879e-15,  3.75253136e-16,
         8.59858557e-15,  2.24919949e-15,  7.72879569e-15,
        -1.52688821e-14,  1.34146660e-15,  2.92371451e-15,
         1.09559282e-15],
       [-1.43906050e-15,  5.98181553e-15,  2.91438581e-14,
        -9.21490340e-15,  8.70340543e-15,  4.96881543e-15,
        -6.16087083e-15,  5.82715711e-14,  2.48652347e-15,
        -3.69468149e-15, -1.65116254e-13,  1.67895243e-15,
        -7.05225300e-15,  9.87490547e-15,  6.05926298e-16,
         4.84515013e-15],
       [-7.22463158e-16, -1.63773870e

In [17]:
(analytical_grad_ic.reshape(7,-1)-grad_ic)/grad_ic

/tmp/ipykernel_2272366/645163605.py:1: RuntimeWarning: invalid value encountered in divide
  (analytical_grad_ic.reshape(7,-1)-grad_ic)/grad_ic


array([[-1.98237603e-15, -4.54268912e-15, -2.09746175e-15,
        -3.00644529e-15, -2.44300663e-15, -2.16064559e-15,
        -7.17652410e-15],
       [ 7.58070534e-16,  0.00000000e+00,  1.03826200e-15,
         2.48568838e-16,  1.94039626e-16,  8.79354698e-16,
         6.92206989e-16],
       [ 5.87797071e-15,  2.51808028e-15,  3.42074060e-15,
         3.15210953e-15,  5.05711220e-15, -1.19865474e-14,
         3.03862813e-15],
       [-9.49738811e-16, -3.19665839e-16, -6.30886643e-16,
        -6.57662469e-16, -1.30941777e-15, -1.25272326e-15,
         9.65244127e-15],
       [-5.66400788e-15, -4.36447148e-15, -4.68528748e-15,
        -4.90517886e-15, -6.65713627e-15, -6.41608496e-15,
        -4.05238671e-15],
       [-3.62381965e-15, -3.27103085e-15, -1.27227443e-16,
        -3.45287876e-15, -4.20990608e-15, -1.82171774e-15,
        -4.44836589e-15],
       [            nan,             nan,             nan,
                    nan,             nan,             nan,
        -4.4408921

# Now the backward part


In [18]:
# Here we compile the two functions for the two gradients
xf_bck = gradients_bck_heyoka(xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, throttles_vars, tof_var, veff)

dtens_ic_bck = hy.diff_tensors(xf_bck,
                    diff_args=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var],
                    diff_order=1
                    )
dtens_bck = hy.diff_tensors(xf_bck,
                    diff_args=[*throttles_vars, tof_var],
                    diff_order=1
                    )
jac_ic_bck = dtens_ic_bck.jacobian   
jac_bck = dtens_bck.jacobian   

d_cf_bck = hy.make_cfunc(jac_bck.flatten(),
                       # Specify the order in which the input
                       # variables are passed to the compiled
                       # function.
                       vars=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, *throttles_vars, tof_var])

d_cf_ic_bck = hy.make_cfunc(jac_ic_bck.flatten(),
                       # Specify the order in which the input
                       # variables are passed to the compiled
                       # function.
                       vars=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, *throttles_vars, tof_var])

In [19]:
analytical_grad_ic_bck = d_cf_ic_bck([rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms] + throttles_bck.tolist() + [tof])
analytical_grad_bck = d_cf_bck([rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms] + throttles_bck.tolist() + [tof])

In [21]:
grad_bck, grad_ic_bck = gradients_bck(rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms, throttles_bck, tof, veff)

In [22]:
(analytical_grad_bck.reshape(7,-1)- grad_bck)/grad_bck

array([[ 5.91109796e-16, -4.00372778e-16, -2.03144637e-16,
        -1.51041573e-16,  1.48789559e-15, -2.06935554e-15,
         6.22546119e-16, -2.03641027e-15, -2.35628415e-15,
        -0.00000000e+00,  3.83838315e-15,  1.87332050e-14,
        -2.02912508e-15, -1.10293302e-14, -8.23363140e-14,
         9.47750345e-16],
       [-2.54768834e-15, -6.20382210e-15, -5.13038942e-15,
        -3.42188355e-15, -5.42804866e-16, -4.37966800e-13,
        -3.41406290e-15, -1.44119644e-15, -3.92755680e-15,
        -7.86349767e-15, -1.32086958e-15, -5.78670222e-15,
         1.26017526e-13, -1.10615095e-15, -4.21732073e-15,
        -7.65762775e-15],
       [-7.82318922e-15, -3.02035692e-14,  5.02699005e-16,
        -4.77084466e-15, -1.93405821e-15,  6.38905851e-15,
        -3.64493077e-15, -3.81188746e-15,  2.22427945e-15,
        -4.53375715e-15, -6.76861369e-15,  8.76397901e-16,
         1.22722963e-14, -1.08883000e-14, -4.34520916e-16,
        -4.83022727e-14],
       [-5.30219046e-14, -1.66709546e

In [23]:
(analytical_grad_ic_bck.reshape(7,-1)- grad_ic_bck)/ grad_ic_bck

/tmp/ipykernel_2272366/3054604412.py:1: RuntimeWarning: invalid value encountered in divide
  (analytical_grad_ic_bck.reshape(7,-1)- grad_ic_bck)/ grad_ic_bck


array([[-4.15657024e-16, -2.78066223e-15,  3.83325739e-16,
        -1.52104989e-15, -1.75791037e-15, -1.24551930e-15,
         4.06031192e-16],
       [-4.48661944e-15, -1.52848899e-15, -1.58319212e-15,
        -3.50356600e-15, -7.88085108e-15, -6.67658088e-15,
        -5.37974076e-15],
       [-1.12290263e-14, -9.82895490e-16, -1.22440105e-14,
        -1.00879446e-14, -5.15376316e-14,  1.33365089e-15,
        -3.48588287e-15],
       [-1.43463386e-14,  3.64986308e-15, -7.23618737e-13,
         4.62278065e-15, -1.44135703e-14, -1.63629696e-14,
        -4.03408449e-14],
       [-3.97818355e-15, -8.61095785e-15, -3.58042427e-15,
        -8.70666210e-15, -7.28655133e-15, -6.03017517e-15,
        -5.65303569e-15],
       [-5.82456599e-15,  1.83100307e-14, -1.86342293e-15,
        -1.01952599e-14, -7.97174957e-15, -3.11740196e-15,
        -4.54229588e-15],
       [            nan,             nan,             nan,
                    nan,             nan,             nan,
        -3.3306690

In [62]:
analytical_grad_ic_bck.reshape(7,7)

array([[-1.28208359e+01, -5.58971967e-01,  1.15851654e+00,
        -4.89831442e+06, -7.62896977e+07,  1.49547674e+07,
         7.16788046e+04],
       [ 8.71031096e+00,  1.59797727e+00, -9.81758448e-01,
         1.11644959e+07,  4.72701522e+07, -8.36945668e+06,
        -4.05742094e+04],
       [-5.43789504e-01, -2.25908662e-01,  2.06285136e-01,
        -1.10784419e+06, -1.23332726e+06,  6.98325610e+06,
         8.08814776e+03],
       [-4.13292256e-07,  1.16036263e-07,  7.38453344e-10,
         6.00408666e-01, -2.46483945e+00,  4.34238254e-01,
         7.74029960e-04],
       [-1.80981595e-06, -1.44476336e-07,  1.47858341e-07,
        -1.12212493e+00, -9.75142703e+00,  1.95157914e+00,
         7.51821278e-03],
       [ 3.54471529e-07, -9.21597828e-09, -1.70459078e-07,
         1.87845601e-01,  2.06119129e+00, -1.89197925e-02,
        -1.62309435e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         9.9999999

In [24]:
analytical_grad_bck.reshape(7,-1)

array([[-1.61336240e+09, -2.38196593e+09,  5.86819771e+08,
        -1.57849639e+09,  3.20477567e+08,  6.12073989e+07,
        -4.78716701e+08,  2.04886274e+08, -2.37150322e+07,
        -2.73800931e+08,  1.45580450e+07, -2.18746303e+06,
        -1.02810940e+08, -7.38854707e+05,  1.45985184e+05,
         1.91927063e+04],
       [ 1.49731708e+09,  1.57566764e+09, -3.71774631e+08,
         1.74186655e+09, -8.78468835e+08, -4.14129727e+05,
         3.31712767e+08, -9.09870539e+08,  8.34680600e+07,
         1.94235324e+07, -4.06127760e+08,  2.38193942e+07,
        -7.31651681e+05, -1.07769459e+08,  9.93747419e+05,
        -9.02650267e+03],
       [-1.25713007e+08, -5.52560541e+07,  2.37138503e+08,
        -1.99896325e+08,  1.15569126e+08, -1.11950100e+08,
        -4.70141587e+07,  9.38191046e+07, -3.21567390e+08,
        -3.08129398e+06,  2.42165945e+07, -2.72043759e+08,
         1.44661897e+05,  9.94335654e+05, -1.02879935e+08,
         9.06156792e+01],
       [-1.38699607e+01, -6.56372620e